### Get Two Nearest Features for Each Grid Centroid
This notebook is the second notebook in the `Phase 3: Transmission Modelling` workflow. We iterate through each grid dataset, and each transmission line dataset, and generate a 'Near Table' containing the coordinates, distance, and feature ID for the closest two transmission lines to each grid cell centroid. The relationships established in this notebook become the basis for the least cost path analysis in part 3.  

In [71]:
import arcpy 
import os
from arcpy import env
from arcpy.sa import *

#### Define Useful Functions

In [72]:
def getCentroid(grid_fc, analysis_fc): # we will only use wind cpa because that excludes less land

    # get name of input grid
    grid_name = os.path.splitext(os.path.basename(grid_fc))[0]

    # Select all grid cells that overlap with our CPAs
    grid_cells = arcpy.management.SelectLayerByLocation(grid_fc, 'HAVE_THEIR_CENTER_IN', 
                                                          analysis_fc, 0, 
                                                          'NEW_SELECTION') # we will only use wind cpa because that excludes less land
    
    cells_copied = arcpy.management.CopyFeatures(grid_cells, 'in_memory\\{}_selected'.format(grid_name))

    # Set ouput feature class name
    output_name = "{}_centroid".format(grid_name) 
    output_path = os.path.join(centroid_workspace, output_name)

    # Convert polygon feature class to point feature class, with the point representing the CENTROID of the feature
    centroid_fc = arcpy.management.FeatureToPoint(cells_copied, output_path, "CENTROID")

    print(f'{output_path}')
    print("centroid saved to {}".format(output_path))

In [73]:
def getFeatureClasses(input_workspace):
    previous_workspace = arcpy.env.workspace
    try:
        arcpy.env.workspace = input_workspace
        feature_class_list = arcpy.ListFeatureClasses()
        return feature_class_list
    finally:
        arcpy.env.workspace = previous_workspace

#### Set Input and Output Paths

In [74]:
# Paths to grids 
cesm2_grid = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\grids.gdb\\cesm2_base_grid"
era5_grid = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\grids.gdb\\era5_base_grid"
analysis_fc = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\scratch.gdb\\cb_2018_CONUS_500k_dissolve"

# Set output workspace and environment
mainOutputFolder = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\"
centroid_workspace = os.path.join(mainOutputFolder, "scratch.gdb")
arcpy.env.workspace = centroid_workspace
arcpy.env.overwriteOutput = True

#### Create Grid Centroid

In [75]:
# Get list for grids 
grid_list = [cesm2_grid]

In [76]:
# Get the centroid 
for grid in grid_list:
    getCentroid(grid, analysis_fc)

C:\Users\Zachary\ASSET\Transmission\analysis\data\scratch.gdb\cesm2_base_grid_centroid
centroid saved to C:\Users\Zachary\ASSET\Transmission\analysis\data\scratch.gdb\cesm2_base_grid_centroid


In [77]:
# Look at the fields of the created feature class
sample_fc = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\scratch.gdb\\cesm2_base_grid_centroid"
fields = [field.name for field in arcpy.ListFields(sample_fc)]
with arcpy.da.SearchCursor(sample_fc, fields) as cursor:
    for row in cursor:
        print(row)

(1, (-2071478.6655000001, 3037687.7782000005), 66.0, 1)
(2, (-2101484.8751, 2937865.2936000004), 67.0, 2)
(3, (-2131578.3709999993, 2837752.429199999), 68.0, 3)
(4, (-2161747.3067000005, 2737388.5950000007), 69.0, 4)
(5, (-2191980.3577999994, 2636811.4639999997), 70.0, 5)
(6, (-2222266.6927000005, 2536057.0702), 71.0, 6)
(7, (-2252595.9452, 2435159.899599999), 72.0, 7)
(8, (-2282958.187999999, 2334152.9771), 73.0, 8)
(9, (-2313343.9088000003, 2233067.947899999), 74.0, 9)
(10, (-2009350.1099999994, 2910829.375), 99.0, 10)
(11, (-2038124.2261999995, 2810329.3539000005), 100.0, 11)
(12, (-2066970.4748, 2709577.3923000004), 101.0, 12)
(13, (-2095878.0277999993, 2608611.3092), 102.0, 13)
(14, (-2124836.5285, 2507467.2777999993), 103.0, 14)
(15, (-2153836.0650999993, 2406179.9174000006), 104.0, 15)
(16, (-2182867.1456000004, 2304782.3806999996), 105.0, 16)
(17, (-2211920.6746999994, 2203306.4354), 106.0, 17)
(18, (-2240987.931399999, 2101782.5415000003), 107.0, 18)
(19, (-2270060.5482, 20002

In [78]:
# set path to input workspace for transmission lines
transmission_lines = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\TransmissionLines.gdb"

In [79]:
voltage_class_list = getFeatureClasses(transmission_lines)
voltage_class_list

['Electric_Power_Transmission_Lines_project',
 'Electric_Power_Transmission_Lines_active',
 'voltage_100_161',
 'voltage_220_287',
 'voltage_500',
 'voltage_under_100',
 'voltage_dc',
 'voltage_not_available',
 'voltage_345',
 'voltage_735_and_above']

In [80]:
# Get centroid points 
input_grid_list = arcpy.ListFeatureClasses('*cesm2_base_grid_centroid')
input_grid_list

['cesm2_base_grid_centroid']

#### Generate Near Table Analysis

In [82]:
# Loop over input grids and voltage classes
for input_grid in input_grid_list:
    for voltage_class in voltage_class_list:

        # Get the full path for the voltage class
        voltage_class_path = os.path.join(transmission_lines, voltage_class)

        # Set output feature class name
        near_name = "{}_{}_near_analysis_twogeos_v2".format(input_grid, voltage_class)
        near_path = os.path.join(centroid_workspace, near_name)

        # Copy input grid
        arcpy.CopyFeatures_management(input_grid, near_path)

        try:
            # Perform near analysis on copied input grid to find the two nearest geometries
            near_table = arcpy.GenerateNearTable_analysis(near_path, voltage_class_path, 'in_memory\\near_table', search_radius="", location="LOCATION", angle="NO_ANGLE", closest="ALL", closest_count=2)

            # Create a dictionary to track the ranks assigned to each feature
            rank_dict = {}

            # Create all the required fields beforehand
            update_fields = ['NEAR_X_1', 'NEAR_Y_1', 'NEAR_FID_1', 'NEAR_DIST_1',
                             'NEAR_X_2', 'NEAR_Y_2', 'NEAR_FID_2', 'NEAR_DIST_2']
            existing_fields = [field.name for field in arcpy.ListFields(near_path)]

            for field_name in update_fields:
                if field_name not in existing_fields:
                    arcpy.AddField_management(near_path, field_name, "DOUBLE")

            # Extract NEAR_X, NEAR_Y, and NEAR_FID from the near table
            with arcpy.da.SearchCursor(near_table, ['IN_FID', 'NEAR_X', 'NEAR_Y', 'NEAR_FID', 'NEAR_DIST', 'NEAR_RANK']) as cursor:
                for row in cursor:
                    in_fid = row[0]
                    near_x = row[1]
                    near_y = row[2]
                    near_fid_value = row[3]
                    near_dist = row[4]
                    near_rank = row[5]

                    # Check if the feature already has a rank of 1
                    if in_fid in rank_dict and rank_dict[in_fid] == 1:
                        # Update the point feature class with NEAR_X_2, NEAR_Y_2, NEAR_FID_2, and NEAR_DIST_2
                        update_fields = ['NEAR_X_2', 'NEAR_Y_2', 'NEAR_FID_2', 'NEAR_DIST_2']
                    else:
                        # Update the point feature class with NEAR_X_1, NEAR_Y_1, NEAR_FID_1, and NEAR_DIST_1
                        update_fields = ['NEAR_X_1', 'NEAR_Y_1', 'NEAR_FID_1', 'NEAR_DIST_1']

                        # Mark this feature as having rank 1
                        rank_dict[in_fid] = 1

                    with arcpy.da.UpdateCursor(near_path, update_fields, f'OBJECTID = {in_fid}') as update_cursor:
                        for update_row in update_cursor:
                            update_row[0] = near_x
                            update_row[1] = near_y
                            update_row[2] = near_fid_value
                            update_row[3] = near_dist
                            update_cursor.updateRow(update_row)

            # Clean up the near table
            arcpy.Delete_management(near_table)

            # Get geoprocessing messages
            print(arcpy.GetMessages())

        except arcpy.ExecuteError:
            print(arcpy.GetMessages(2))
            
        except Exception as err:
            print(err.args[0])

Start Time: Tuesday, October 24, 2023 8:59:11 PM
Succeeded at Tuesday, October 24, 2023 8:59:11 PM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, October 24, 2023 8:59:18 PM
Succeeded at Tuesday, October 24, 2023 8:59:18 PM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, October 24, 2023 8:59:24 PM
Succeeded at Tuesday, October 24, 2023 8:59:24 PM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, October 24, 2023 8:59:28 PM
Succeeded at Tuesday, October 24, 2023 8:59:28 PM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, October 24, 2023 8:59:33 PM
Succeeded at Tuesday, October 24, 2023 8:59:33 PM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, October 24, 2023 8:59:38 PM
Succeeded at Tuesday, October 24, 2023 8:59:38 PM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, October 24, 2023 8:59:42 PM
Succeeded at Tuesday, October 24, 2023 8:59:42 PM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, October 24, 2023 8:59:46 PM
Succeeded at Tuesday, October 24, 2023 8:59:46 PM (Elap